In [1]:
import os
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
import pandas as pd
import numpy as np
import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col
from pyspark.sql import SQLContext
from pyspark.ml.feature import Word2Vec
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from gensim.models import Word2Vec

from nltk.tokenize import word_tokenize
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, Word2Vec, StringIndexer, IndexToString, StopWordsRemover
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator,BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from gensim.models import Word2Vec
import numpy as np
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, StopWordsRemover, Word2Vec
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


/home/hadi/.local/lib/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
os.getcwd()
os.chdir('/home/hadi/Datasets')

Without Distributed Computing/Without PySpark


In [3]:
df_undistributed=pd.read_csv('IMDB Dataset.csv')
df_undsitributed_train=df_undistributed.iloc[:40000,:]
df_undsitributed_test=df_undistributed.iloc[10000:,:]


words=[',','<','>','?','br','/','.','-',';',':','(',')','[',']','{','}','!','@','#','$','%','^','&','*','_','+','=','|','\\','"',"'",'<br,','<br','<br>','<br/>','<br />','<br >','<br /']
lemmatizer = WordNetLemmatizer()

stop_words = set(stopwords.words('english'))
stop_words.add('.')
stop_words.add('<')
stop_words.add('>')
stop_words.add('?')
stop_words.add('br')
stop_words.add('/')
stop_words.add('.')
stop_words.add('-')
stop_words.add(';')

for i in range(len(df_undistributed['review'])):
	word_tokens = word_tokenize(df_undistributed['review'][i])
	filtered_sentence = [w for w in word_tokens if not w in stop_words and w != ',']
	filtered_sentence = []
	for w in word_tokens:
		if w not in stop_words and w != ',':
			lemmatizer.lemmatize(w)
			filtered_sentence.append(w)
	df_undistributed['review'][i] = ' '.join(filtered_sentence)
	df_undistributed['review'][i] = df_undistributed['review'][i].lower()


tokenized_reviews = [word_tokenize(review) for review in df_undistributed['review']]







/tmp/ipykernel_3582/2999134276.py:28: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_undistributed['review'][i] = ' '.join(filtered_sentence)
/tmp/ipykernel_3582/2999134276.py:29: FutureWarning: ChainedAssignmentError: behaviour will chang

In [4]:
from gensim.models import Word2Vec
import numpy as np
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

word2vec_model = Word2Vec(sentences=tokenized_reviews, vector_size=30, window=5, min_count=1, workers=4)

def document_vector(word2vec_model, doc):
    doc = [word for word in doc if word in word2vec_model.wv.key_to_index]
    if len(doc) == 0:
        return np.zeros(word2vec_model.vector_size)
    return np.mean(word2vec_model.wv[doc], axis=0)  

X = [document_vector(word2vec_model, doc) for doc in tokenized_reviews]
y = df_undistributed['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

RF = RandomForestClassifier(max_depth=4)
RF.fit(X_train, y_train)

y_pred = RF.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print('F1:', f1_score(y_test, y_pred, average='weighted'))

text = 'I love this movie, But the title is not good overall the movie was lovely'
text = word_tokenize(text)
text = [document_vector(word2vec_model, text)]
print(RF.predict(text))
print(RF.predict_proba(text))

text_2 = 'I hate this movie'
text_2 = word_tokenize(text_2)
text_2 = [document_vector(word2vec_model, text_2)]
print(RF.predict(text_2))
print(RF.predict_proba(text_2))

text_3 = 'The movie was very strong and powerful'
text_3 = word_tokenize(text_3)
text_3 = [document_vector(word2vec_model, text_3)]
print(RF.predict(text_3))
print(RF.predict_proba(text_3))


Accuracy: 0.7644
F1: 0.7644010178447651
['positive']
[[0.41638561 0.58361439]]
['negative']
[[0.50273261 0.49726739]]
['positive']
[[0.48467448 0.51532552]]


DISTRIBUTED WORD EMBEDDINGS

In [5]:
spark=SparkSession.builder \
    .appName("Spark NLP") \
    .master("spark://172.28.111.195:7077") \
    .config("spark.driver.memory", "16G") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryoserializer.buffer.max", "2000M") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.3.3") \
    .getOrCreate()


your 131072x1 screen size is bogus. expect trouble
24/05/04 02:56:39 WARN Utils: Your hostname, Hadi resolves to a loopback address: 127.0.1.1; using 172.28.111.195 instead (on interface eth0)
24/05/04 02:56:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/hadi/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hadi/.ivy2/cache
The jars for the packages stored in: /home/hadi/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-efad3dc6-6c7e-4171-8898-67ad4db26d75;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.3.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in central
	f

In [6]:
df=pd.read_csv('IMDB Dataset.csv')
df.head()
df_train=df[:20000]
df_test=df[20000:]
spark_df=spark.createDataFrame(df)
spark_df_train,spark_df_test=spark_df.randomSplit([0.8,0.2],seed=42)

In [7]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, StopWordsRemover, Word2Vec
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

words = [',', '<', '>', '?', 'br', '/', '.', '-', ';', ':', '(', ')', '[', ']', '{', '}', '!', '@', '#', '$', '%', '^', '&', '*', '_', '+', '=', '|', '\\', '"', "'", '<br,', '<br', '<br>', '<br/>', '<br />', '<br >', '<br /']
tokenizer = Tokenizer(inputCol="review", outputCol="words")

remover = StopWordsRemover(inputCol="words", outputCol="filtered", stopWords=words)

word2vec = Word2Vec(vectorSize=30, minCount=1, windowSize=5, inputCol=remover.getOutputCol(), outputCol="features")

labelIndexer = StringIndexer(inputCol="sentiment", outputCol="label").fit(spark_df_train)

rf = RandomForestClassifier(numTrees=100, maxDepth=4, labelCol="label", featuresCol="features")

labelConverter = IndexToString(inputCol="prediction", outputCol="predicted_sentiment", labels=labelIndexer.labels)

pipeline = Pipeline(stages=[tokenizer, remover, word2vec, labelIndexer, rf, labelConverter])

model = pipeline.fit(spark_df_train)

prediction = model.transform(spark_df_test)

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(prediction)
print("Accuracy:", accuracy)

text_spark = 'I love this movie, But the title is not good overall the movie was lovely'
text_spark_2 = 'I hate this movie'
text_spark_3 = 'The movie was very strong and powerful'

text_data = [text_spark, text_spark_2, text_spark_3]
text_data = pd.DataFrame(text_data, columns=['review'])
text_data = spark.createDataFrame(text_data)

prediction = model.transform(text_data)
prediction.show()


24/05/04 02:57:17 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/05/04 02:57:32 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/05/04 02:57:47 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/05/04 02:58:02 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/05/04 02:58:16 WARN TaskSetManager: Stage 0 contains a task of very large size (31878 KiB). The maximum recommended task size is 1000 KiB.
24/05/04 02:58:25 WARN TaskSetManager: Stage 3 contains a task of very large size (31878 KiB). The maximum recommended task size is 1000 KiB.
24/05/04 02:

Accuracy: 0.6904592923045726


24/05/04 03:01:13 WARN StringIndexerModel: Input column sentiment does not exist during transformation. Skip StringIndexerModel for this column.


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|              review|               words|            filtered|            features|       rawPrediction|         probability|prediction|predicted_sentiment|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|I love this movie...|[i, love, this, m...|[i, love, this, m...|[0.13165435185655...|[46.4435573976831...|[0.46443557397683...|       1.0|           negative|
|   I hate this movie|[i, hate, this, m...|[i, hate, this, m...|[0.27294314932078...|[54.2698277039243...|[0.54269827703924...|       0.0|           positive|
|The movie was ver...|[the, movie, was,...|[the, movie, was,...|[0.13679167335586...|[66.6522349753975...|[0.66652234975397...|       0.0|           positive|
+--------------------+--------------------+---

In [8]:
spark.stop()